In [1]:
import tkinter as tk
from tkinter import messagebox
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import os

class BMICALCULATORAPP:
    def __init__(self, root):
        self.root = root
        self.root.title("BMI CALCULATOR WITH HISTORY AND TREND")
        self.root.configure(bg="teal")

        self.history_file = "bmi_history.csv"
        if os.path.exists(self.history_file):
            self.history = pd.read_csv(self.history_file)
        else:
            self.history = pd.DataFrame(columns=["Date", "Weight (Kg)", "Height(cm)", "BMI"])

        self.create_widgets()

    def create_widgets(self):
        # Labels & Entries
        tk.Label(self.root, text="WEIGHT (KG):", bg="teal", fg="white", font=('Arial', 10, 'bold')).grid(row=0, column=0, sticky="e", pady=2)
        self.weight_entry = tk.Entry(self.root)
        self.weight_entry.grid(row=0, column=1, pady=2)

        tk.Label(self.root, text="HEIGHT (CM):", bg="teal", fg="white", font=('Arial', 10, 'bold')).grid(row=1, column=0, sticky="e", pady=2)
        self.height_entry = tk.Entry(self.root)
        self.height_entry.grid(row=1, column=1, pady=2)

        # Styled Buttons
        btn_style = {'bg': 'white', 'fg': 'brown', 'font': ('Arial', 10, 'bold'), 'activebackground': 'lightgray'}

        tk.Button(self.root, text="CALCULATE BMI", command=self.calculate_bmi, **btn_style).grid(row=2, column=0, columnspan=2, pady=5)
        tk.Button(self.root, text="SHOW HISTORY", command=self.show_history, **btn_style).grid(row=3, column=0, columnspan=2, pady=5)
        tk.Button(self.root, text="PLOT TREND", command=self.plot_trends, **btn_style).grid(row=4, column=0, columnspan=2, pady=5)

        # Output Labels
        self.result_label = tk.Label(self.root, text="BMI: --", bg="teal", fg="white", font=('Arial', 12, 'bold'))
        self.result_label.grid(row=5, column=0, columnspan=2, pady=5)

        self.bmi_category_label = tk.Label(self.root, text="", bg="teal", fg="white", font=('Arial', 10, 'italic'))
        self.bmi_category_label.grid(row=6, column=0, columnspan=2, pady=5)

    def calculate_bmi(self):
        weight = self.weight_entry.get()
        height = self.height_entry.get()

        if not weight or not height:
            messagebox.showerror("INPUT ERROR", "Please enter weight and height.")
            return

        try:
            weight = float(weight)
            height = float(height) / 100  # convert cm to meters
            if weight <= 0 or height <= 0:
                raise ValueError
        except ValueError:
            messagebox.showerror("INPUT ERROR", "Please enter valid positive numbers.")
            return

        bmi = weight / (height ** 2)
        category = self.get_bmi_category(bmi)

        self.result_label.config(text=f"BMI: {bmi:.2f}")
        self.bmi_category_label.config(text=f"Category: {category}")
        self.save_history(weight, height * 100, bmi)

    def get_bmi_category(self, bmi):
        if bmi < 18.5:
            return "Underweight (Danger)"
        elif 18.5 <= bmi < 24.9:
            return "Normal (Safe)"
        elif 25 <= bmi < 29.9:
            return "Overweight (Warning)"
        else:
            return "Obese (Danger)"

    def save_history(self, weight, height, bmi):
        current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        new_row = pd.DataFrame([{
            "Date": current_time,
            "Weight (Kg)": weight,
            "Height(cm)": height,
            "BMI": bmi
        }])
        self.history = pd.concat([self.history, new_row], ignore_index=True)
        self.history.to_csv(self.history_file, index=False)

    def show_history(self):
        try:
            self.history = pd.read_csv(self.history_file)
            if self.history.empty:
                messagebox.showinfo("BMI HISTORY", "No history available.")
                return
            history_text = self.history.to_string(index=False)
            messagebox.showinfo("BMI HISTORY", history_text)
        except Exception as e:
            messagebox.showerror("ERROR", f"Unable to read history.\n{e}")

    def plot_trends(self):
        try:
            self.history = pd.read_csv(self.history_file)
            if self.history.empty:
                messagebox.showinfo("BMI TREND", "No data to plot.")
                return

            plot_window = tk.Toplevel(self.root)
            plot_window.title("BMI Trend Plot")

            fig, ax = plt.subplots(figsize=(8, 4))
            ax.plot(pd.to_datetime(self.history["Date"]), self.history["BMI"],
                    marker='o', linestyle='-', color='blue')
            ax.set_title("BMI Trend Over Time")
            ax.set_xlabel("Date")
            ax.set_ylabel("BMI")
            ax.grid(True)
            fig.autofmt_xdate()

            canvas = FigureCanvasTkAgg(fig, master=plot_window)
            canvas.draw()
            canvas.get_tk_widget().pack()

        except Exception as e:
            messagebox.showerror("ERROR", f"Unable to plot trend.\n{e}")

# Run the app
if __name__ == "__main__":
    root = tk.Tk()
    app = BMICALCULATORAPP(root)
    root.mainloop()
